The goal is to process the raw EEG signal
- Load the file from Data/BNM (N11....csv is in byte format and UnicornRecorder is already in float values)
- Plot the raw EEG signal
- Apply bandpass filter (0.5Hz - 30Hz) and then notch filter to get rid of powerline noise (50V)
- Plot the filtered EEG signal
- Calculate and plot Power Spectral Density of the signal
- Apply Butterworth filter of order 2 to divide into following bands: 
    - Delta 1-4 Hz
    - Theta 4-8 Hz
    - Alpha 8-12 Hz
    - Beta Low 12-16 Hz
    - Beta Mid 16-20 Hz
    - Beta High 20-30 Hz
    - Gamma 30-50 Hz
- Plot all the bands

Extra info:
1. If you see weird signal on diagram, play a bit with scaling first, the data is checked and works properly
2. You may consider to cut out start of the signal as it has a lot of artifcats

If you would like to play a bit more, try experimenting with:
- different params and filtering methods
- improve your plots
- try out the CSP
- try out frequency-time domain like wavelet transformation
- create a spectrogram 


In [ ]:
import struct

import numpy as np
import mne


# Funkcja do przetwarzania pliku binarnego N11.csv
def process_binary_file(file_path, scale_factor=0.001, num_channels=8):
    """
    Przetwarza plik binarny z danymi EEG.

    :param file_path: Ścieżka do pliku binarnego.
    :param scale_factor: Współczynnik skalowania do przekształcania wartości na µV.
    :param num_channels: Liczba kanałów EEG.
    :return: Dane jako tablica NumPy (n_channels x n_samples).
    """
    with open(file_path, 'rb') as file:
        decoded_data = []
        while True:
            byte_data = file.read(68)
            if not byte_data:
                break
            try:
                message = struct.unpack('17f', byte_data)
            except struct.error:
                print(f"Nie można odczytać danych z pliku: {file_path}")
                raise ValueError("Błąd dekodowania danych")
            decoded_data.append(message[:num_channels])

    # Przekształcenie na tablicę NumPy i skalowanie
    eeg_data = np.array(decoded_data, dtype=np.float32)

    # # Formatowanie danych jako (n_channels x n_samples)
    # n_samples = len(eeg_data) // num_channels
    # eeg_data = eeg_data[:n_samples * num_channels].reshape(num_channels, n_samples)

    return eeg_data.T

# Funkcja do tworzenia obiektu MNE RawArray
def create_mne_raw(data, sfreq, ch_names):
    """
    Tworzy obiekt MNE RawArray z danych.

    :param data: Tablica NumPy z danymi (kanały x próbki).
    :param sfreq: Częstotliwość próbkowania (Hz).
    :param ch_names: Nazwy kanałów (lista stringów).
    :return: Obiekt mne.io.RawArray.
    """
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types="eeg")
    raw = mne.io.RawArray(data, info)
    raw.crop(tmin=1000, tmax=1050)
    return raw

# Przetwarzanie pliku N11.csv
n11_file_path = 'Data/BNM/N11_TetrisExperiment_2024-03-06_17-54.csv'
sfreq_n11 = 250  # Częstotliwość próbkowania dla N11.csv
n_channels_n11 = 8  # liczba kanałów

try:
    n11_data = process_binary_file(n11_file_path, num_channels=n_channels_n11)
    ch_names_n11 = [f"EEG {i + 1}" for i in range(n_channels_n11)]  # Nazwy kanałów
    raw_n11 = create_mne_raw(n11_data, sfreq=sfreq_n11, ch_names=ch_names_n11)
    print("Dane N11.csv zostały pomyślnie przekształcone na MNE RawArray.")
except Exception as e:
    print(f"Błąd przetwarzania N11.csv: {e}")
    raw_n11 = None

# Wizualizacja za pomocą MNE
if raw_n11 is not None:
    raw_n11.plot(duration=1, scalings='800', title="Dane EEG N11 (MNE)")

# Zastosowanie filtra pasmowo-przepustowego do RawArray
raw_filtered = raw_n11.copy()  # Tworzymy kopię danych, aby zachować oryginał
raw_filtered.notch_filter(freqs=50.0)
raw_filtered.filter(l_freq=0.5, h_freq=30.0)  # Filtrujemy w zakresie 0.5–30 Hz

# Wizualizacja przefiltrowanego sygnału
raw_filtered.plot(duration=20, scalings='300', title="Dane EEG N11 (Filtr pasmowo-przepustowy 0.5–30 Hz)")

# Wykres PSD dla wszystkich kanałów
raw_filtered.plot_psd(fmin=0.5, fmax=40.0, average=False, dB=True)

# Rytm delta (1–4 Hz)
raw_delta = raw_filtered.copy()
raw_delta.filter(l_freq=1, h_freq=4.0, method='iir')
raw_delta.plot(duration=30, scalings='200', title="Rytm delta (1–4 Hz)")
raw_delta.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm theta (4–8 Hz)
raw_theta = raw_filtered.copy()
raw_theta.filter(l_freq=4.0, h_freq=8.0, method='iir')
raw_theta.plot(duration=10, scalings='100', title="Rytm theta (4–8 Hz)")
raw_theta.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm alfa (8–12 Hz)
raw_alpha = raw_filtered.copy()
raw_alpha.filter(l_freq=8.0, h_freq=12.0, method='iir')
raw_alpha.plot(duration=10, scalings='20', title="Rytm alfa (8–12 Hz)")
raw_alpha.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm beta niski (12-16 Hz)
raw_beta_low = raw_filtered.copy()
raw_beta_low.filter(l_freq=12.0, h_freq=16.0, method='iir')
raw_beta_low.plot(duration=10, scalings='15', title="Rytm beta niski (8–12 Hz)")
raw_beta_low.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm beta średni (16–20 Hz)
raw_beta_mid = raw_filtered.copy()
raw_beta_mid.filter(l_freq=16.0, h_freq=20.0, method='iir')
raw_beta_mid.plot(duration=7, scalings='20', title="Rytm beta średni (16–20 Hz)")
raw_beta_mid.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm beta wysoki (20–30 Hz)
raw_beta_high = raw_filtered.copy()
raw_beta_high.filter(l_freq=20.0, h_freq=30.0, method='iir')
raw_beta_high.plot(duration=5, scalings='30', title="Rytm beta wysoki (20–30 Hz)")
raw_beta_high.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)

# Rytm gamma (30–50 Hz)
raw_gamma = raw_filtered.copy()
raw_gamma.filter(l_freq=30.0, h_freq=50.0, method='iir')
raw_gamma.plot(duration=3, scalings='15', title="Rytm gamma (30–50 Hz)")
raw_gamma.plot_psd(fmin=0.5, fmax=60.0, average=False, dB=True)